# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from lindblad_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [4]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_shot_noise(targets_spam, shots = shots)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, shots=1024):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm.povm)
    
    #add noise
    targets_map = add_shot_noise(targets_map, shots = shots)

    return inputs_map, targets_map


def model_pipeline(channel_target = None, shots=1024, N_map = None, num_iter_spam=None, num_iter_map=None):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.9, c2=0.9)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, shots=shots)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d, c=0.9),
                      povm = CorruptionMatrix(d, c=0.9),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    #spam_model.pretrain(300, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter_map,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = num_iter_map,
                N = 500,
                )
    #################################################################################
    model.optimizer = None
    spam_model.optimizer = None
    model.inputs = None
    model.targets = None
    model.inputs_val = None
    model.targets_val = None

    return model

## Fit Models

In [11]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n

channel_target1 = CompactLindbladMap(d, 1, 1, 0.1)
model1 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)

channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
model2 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)

channel_target3 = CompactLindbladMap(d, 16, 100, 0.001)
model3 = model_pipeline(channel_target3, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)

channel_target4 = CompactLindbladMap(d, 8, 1, 0.01)
model4 = model_pipeline(channel_target4, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)


saver([[channel_target1, channel_target2, channel_target3, channel_target4],
       [model1, model2, model3, model4]], data_path("atypical_maps_benchmark.model"))

#saver([[channel_target1],
#       [model1]], data_path("atypical_maps_benchmark.model"))

  0%|          | 0/4000 [00:00<?, ?it/s]

0.000730815239418286


  0%|          | 0/4000 [00:00<?, ?it/s]

0.2522026107275442 -0.09635478056450521
0.05936058180636232 -0.3899705250785339
0.016261923116314792 -0.6904450944477275
0.009365052591266677 -0.8129878109859814
0.007762401064528082 -0.8654267614275218
0.007186431661177628 -0.8928265321477441
0.006916717768014154 -0.9088602794746254
0.006768724127022852 -0.9189371684621234
0.006671735895679792 -0.9255630174739587
0.006615801000310474 -0.9302060992209514
0.006576474536503826 -0.9335778489271783
0.00655176934676036 -0.9357606370871944
0.006535662531859102 -0.9374909800907267
0.006515655678077449 -0.9387211735220728
0.006506594122375152 -0.9395756680998879
0.0065017912150029415 -0.9400727509684758
0.006497470585141921 -0.9409270950863449
0.006493601052208069 -0.9410619250810395
0.006491166247954405 -0.9414947889210017
0.00649043213359536 -0.9419562917244203
0.00649509484565745 -0.9414184995284446
0.006484355787051177 -0.9421083969975833
0.006484693779663787 -0.9417194492318975
0.006495306774060401 -0.9424314337642704
0.006483493430371469

  0%|          | 0/4000 [00:00<?, ?it/s]

...

0.5211159850447601 -0.15669000604136157
0.14813877640778364 -0.3992688046143136
0.01742496068516106 -0.7121539659858652
0.009231498587211123 -0.7961793835743612
0.007706733937946242 -0.830744861993646
0.006964298894504138 -0.8532123912161523
0.006565318494669901 -0.8685919368772671
0.006328676322916 -0.879543553571796
0.006173875526167471 -0.8873975788106835
0.0060762370581396025 -0.8930387110332754
0.006001920780123309 -0.8969022721568153
0.005949640547395266 -0.9001185779156153
0.005905078922196703 -0.9021080935045697
0.005872528446636266 -0.9031076491989874
0.005851713986298481 -0.904414562265188
0.005843797448774662 -0.9050795024872024
0.005820466085984666 -0.9053728077197392
0.0058080430157140305 -0.9057995255267361
0.005799313464530846 -0.9053874820443346
0.005789664342773913 -0.9052980879720652
0.0057928720670707414 -0.9054081344013455
0.005786471676934508 -0.9052564880812464
0.005777889144445512 -0.904251657366044
0.00577099488454235 -0.9042893049632922
0.005779804266272675 -0.

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0007117775890604215


  0%|          | 0/4000 [00:00<?, ?it/s]

0.1184389562354085 -0.4363524927727841
0.01958959034890855 -0.6560336616798985
0.00841117857377542 -0.7962915735434174
0.006513030281092514 -0.8478801311566997
0.005946043620466693 -0.8698838440760821
0.0056850709941934675 -0.8799025383947268
0.005532559460668416 -0.884055998897291
0.005444955415608176 -0.8846284407459553
0.0053865611141588055 -0.8839711195958693
0.0053490585464159935 -0.8818400202531271
0.005318315500280679 -0.8800251803221447
0.005299816254844493 -0.8775779096272748
0.005281721843029412 -0.8756332282100574
0.005271936447145652 -0.8730289739995212
0.005260250961704415 -0.8708941064201952
0.0052555016462694564 -0.8684006499824468
0.005245393910893451 -0.8665827718875441
0.005241722455153038 -0.8648574318806986
0.005245217434414837 -0.8628031259199055
0.005240257666060747 -0.8611751879184903
0.0052435359408000305 -0.8595401233212498
0.005244014523120093 -0.8582416397416486
0.005240165696193896 -0.8566523922298919
0.005243084469937173 -0.8554154508430527
0.00524494599024

## Sample and Noise Sensitivity

In [ ]:
n = 3
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

channel_target = CompactLindbladMap(d, 1, 1, 0.1)
model_ns1 = model_pipeline(channel_target, 
                           shots=256, 
                           N_map=256, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000)


model_ns2 = model_pipeline(channel_target, 
                           shots=1024, 
                           N_map=1024, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000)

model_ns3 = model_pipeline(channel_target, 
                           shots=4096, 
                           N_map=4096, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000)

saver([[channel_target],
       [model_ns1, model_ns2, model_ns3]], data_path("atypical_maps_robustness.model"))